In [1]:
from utils import detect_address
import requests as r
import ed25519
import base64 as b
from array import array
import hashlib

/Users/tvorogme/opt/anaconda3/lib/python3.9/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
DTON_HOST = "https://dton.io"
DTON_GRAPHQL_URL = f'{DTON_HOST}/graphql/'

In [3]:
wallet_address = "0:ac0922e2b0f67ac54b64822fd39f973e922a5e9551d10b119d3eef71d606954d"

query = """
mutation {
  run_method(
    account_search_by_address: {address_friendly: \""""+ wallet_address +"""\"}
    method_name: "get_public_key"
  ) {
    stack {
      value
      value_type
    }
  }
}"""

In [4]:
data = r.post(DTON_GRAPHQL_URL, json={'query': query})

In [5]:
print(data.json())

{'data': {'run_method': {'stack': [{'value': '22904357750873648918539994230726885374799650012723529598629953594018217887363', 'value_type': 'int'}]}}}


In [6]:
pubkey = hex(int(data.json()['data']['run_method']['stack'][0]['value']))[2:]

In [7]:
# example
answer = {"timestamp": 1675102176,
          "domain": {
              "lengthBytes": 31,
              "value": "b4fd-213-87-160-231.eu.ngrok.io"
          },
          "signature": "qJikM1i9wVi2zKss+JpLu3YajTyT54iDZaZNzkR5hphFKxFaL1W97lj+qv0oNaaR3uRYdcKxd7AMU+DMVdMQDQ==",
          "payload": "OQ1UQGOHQW8IODBM8JRD"}


In [8]:
tonProofPrefix = 'ton-proof-item-v2/'
tonConnectPrefix = 'ton-connect'

In [9]:
address_parsed = detect_address(wallet_address)['raw_form'].split(':')

In [10]:
wc = int(address_parsed[0])
wc_str = (wc).to_bytes(4, byteorder='big')
assert int(wc_str.hex(), 16) == wc

In [11]:
ad = int(address_parsed[1].zfill(64), 16)
ad_str = (ad).to_bytes(32, byteorder='big')

In [12]:
ts = int(answer['timestamp'])
ts_str = (ts).to_bytes(8, byteorder='little')

In [13]:
dl = answer['domain']['lengthBytes']
dl_str = (dl).to_bytes(4, byteorder='little')

In [14]:
domain = answer['domain']['value']
domain_str = domain.encode('utf-8')
assert len(domain_str) == dl

In [15]:
payload = answer['payload']
payload_str = payload.encode('utf-8')

In [16]:
message = tonProofPrefix.encode() + \
            wc_str + \
            ad_str + \
            dl_str + \
            domain_str + \
            ts_str + \
            payload_str

In [17]:
tonProofPrefix.encode(), wc_str, ad_str, dl_str, domain_str, ts_str, payload_str

(b'ton-proof-item-v2/',
 b'\x00\x00\x00\x00',
 b'\xac\t"\xe2\xb0\xf6z\xc5Kd\x82/\xd3\x9f\x97>\x92*^\x95Q\xd1\x0b\x11\x9d>\xefq\xd6\x06\x95M',
 b'\x1f\x00\x00\x00',
 b'b4fd-213-87-160-231.eu.ngrok.io',
 b'\xe0\x07\xd8c\x00\x00\x00\x00',
 b'OQ1UQGOHQW8IODBM8JRD')

In [18]:
def get_hash(x):
    h = hashlib.new('sha256')
    h.update(x)
    return bytes.fromhex(h.hexdigest())

In [19]:
h = get_hash(message)
h

b'1\xff\x1aqN\xa3Ip\xe8\x97\xcc\x83\xd2\x01\xce\xcb\xbb9\x80h\xbc\xf3\x8e\xab\x9eL\x9f\xcd)m\x18n'

In [20]:
ff = bytes(bytearray([0xff, 0xff]))
ff, tonConnectPrefix.encode(), h

(b'\xff\xff',
 b'ton-connect',
 b'1\xff\x1aqN\xa3Ip\xe8\x97\xcc\x83\xd2\x01\xce\xcb\xbb9\x80h\xbc\xf3\x8e\xab\x9eL\x9f\xcd)m\x18n')

In [21]:
public_key = ed25519.VerifyingKey(pubkey, 
                                  encoding='hex')

# if fail - will raise exception
public_key.verify(b.b64decode(answer['signature']), 
                  get_hash(ff + tonConnectPrefix.encode() + h))